In [1]:
import numpy as np
from functools import cached_property

In [27]:
class QuantumSystem:
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
        self.uniform_superposition = self.normalise_system(np.array([complex(1, 0)] * 2**(self.n_qubits)))
        self.register = self.uniform_superposition.copy()

    def normalise_system(self, system):
        return system / np.sqrt(np.sum(np.abs(system)**2))

    def normalise(self):
        self.register = self.normalise_system(self.register)

class QuantumSearch(QuantumSystem):
    def __init__(self, n_qubits, oracle):
        self.n_qubits = n_qubits
        self.oracle = oracle
        QuantumSystem.__init__(self, self.n_qubits)

    @cached_property
    def oracle_operator(self):
        a = np.eye(2**self.n_qubits)
        mask = np.array([self.oracle(i) for i in range(2**self.n_qubits)])
        a[mask] *= -1
        return a

    @cached_property
    def diffusion_operator(self):
        s = self.uniform_superposition.flatten()[np.newaxis].T
        return 2 * np.matmul(s, s.T) - np.eye(2**self.n_qubits)
    
    def step(self):
        self.register = np.multiply(np.diag(self.oracle_operator), self.register)
        self.register = np.matmul(self.diffusion_operator, self.register)
        self.normalise()
        
    def measurement(self):
        return np.searchsorted(np.cumsum(np.abs(self.register)**2), np.random.random())

In [80]:
def oracle(x):
    return (x > 2) and (x & (x - 1) == 0) and (x & (x + 18) == 0)

qsearch = QuantumSearch(n_qubits=5, oracle=oracle)

iterations = np.ceil((np.pi / 4) * np.sqrt(qsearch.n_qubits)).astype(int)

for i in range(6):
    qsearch.step()
    print(qsearch.measurement())


25
16
30
16
16
16
